In [1]:
pip install transformers

     |████████████████████████████████| 3.8 MB 14.8 MB/s 
     |████████████████████████████████| 596 kB 77.4 MB/s 
     |████████████████████████████████| 6.5 MB 76.0 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 71.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import os
import json
import random
import numpy as np
import pandas as pd

from tqdm import tqdm

import torch
from torch.optim import Adam, AdamW
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
from transformers import BertTokenizer, BertTokenizerFast, BertForQuestionAnswering
from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  f1_score

import gc

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/My Drive/NLP

/content/gdrive/My Drive/NLP


In [5]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2022-03-23 05:40:26--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.111.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  76.0MB/s    in 0.5s    

2022-03-23 05:40:27 (76.0 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-03-23 05:40:27--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.111.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[=====

In [6]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [7]:
def data_load(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = [] 
    start_ids = []
    end_ids = []

    for datas in squad_dict['data']:
        for paragraphs in datas['paragraphs']:
            context = paragraphs['context']
            for qas in paragraphs['qas']:
                question = qas['question']
                for answer in qas['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer['text'])

                    start_index = answer['answer_start']
                    start_ids.append(start_index)

                    answer['text'] = answer['text'].rstrip()
                    
                    end_index = start_index + len(answer['text'])
                    end_ids.append(end_index)                  

    return pd.DataFrame({'contexts' : contexts, 'questions' : questions, 'answers' : answers, 'start_ids': start_ids,'end_ids': end_ids})

In [8]:
train_df = data_load('KorQuAD_v1.0_train.json')
valid_df = data_load('KorQuAD_v1.0_dev.json')

In [9]:
def qa_preprocess(df, batch_size=16, method='train'):
  if method == 'train' or method == 'valid':    
      batch_input = tokenizer(df['contexts'].tolist(), df['questions'].tolist(), truncation=True, padding=True)

      start_ids = df['start_ids'].tolist()
      end_ids = df['end_ids'].tolist()

      start_positions = [batch_input.char_to_token(i, start_ids[i]) for i in range(len(start_ids))]
      end_positions = [batch_input.char_to_token(i, end_ids[i]-1) for i in range(len(end_ids))]
      deleting_list = [i for i, v in enumerate(end_positions) if v == None]
          
      batch_input.update({'start_positions': start_positions, 'end_positions': end_positions})

     
      batch_input = {key : np.delete(np.array(value), deleting_list, axis=0) for key, value in batch_input.items()}
      batch_input = {key : torch.tensor(value.astype(int)) for key, value in batch_input.items()}

      dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'], batch_input['start_positions'], batch_input['end_positions'])
      if method == 'train':
        dataset_sampler = RandomSampler(dataset)
        dataloader = DataLoader(dataset, sampler=dataset_sampler, batch_size=batch_size)
      elif method == 'valid':
        dataloader = DataLoader(dataset, batch_size=batch_size)
      return dataloader, deleting_list

  elif method == 'test':
      batch_input = tokenizer(df['contexts'].tolist(), df['questions'].tolist(), truncation=True, padding=True)
      batch_input = {key : torch.tensor(value) for key, value in batch_input.items()}

      dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'])
      dataloader = DataLoader(dataset, batch_size=batch_size)

  return dataloader

# model training


In [10]:
def train_one_epoch(optimizer, scheduler, dataloader,accumulation=2):
    model.train()

    running_loss = 0.0
    train_losses = []

    for ids, batchs in enumerate(tqdm(dataloader)):
        batch = tuple(b.to(device) for b in batchs)

        inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
                "start_positions": batch[3],
                "end_positions": batch[4],
            }


        optimizer.zero_grad()

        outputs = model(**inputs)

        (outputs.loss/accumulation).backward()
        running_loss += outputs.loss.item()

        del inputs
        if (ids+1) % accumulation: # enumerate의 ids
            continue

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        train_losses.append(running_loss / accumulation)

    train_loss = sum(train_losses) / len(train_losses)
    
    return train_loss

# model_evaluate

In [11]:
def evaluate_one_epoch(dataloader,accumulation=2):
    model.eval()

    start_preds = []
    end_preds = []
    input = []

    for ids, batchs in enumerate(tqdm(dataloader)):
        batch = tuple(b.to(device) for b in batchs)

        inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2]
            }


        with torch.no_grad():
            outputs = model(**inputs)

        # del inputs
        # if (ids+1) % accumulation:
        #     continue

        # CPU로 데이터 이동
        start_pred = outputs['start_logits'].detach().cpu()
        end_pred = outputs['end_logits'].detach().cpu()

        input.append(inputs['input_ids'].detach().cpu())
        start_preds.append(start_pred)
        end_preds.append(end_pred)

    input = torch.cat(input, dim=0).tolist()
    start_preds = torch.cat(start_preds, dim=0).argmax(dim=-1).tolist()
    end_preds = torch.cat(end_preds, dim=0).argmax(dim=-1).tolist()

    answer = [tokenizer.decode(input[s:e+1]) for input, s, e in zip(input,start_preds,end_preds)]

    return answer

# QA Model

In [12]:
def qa_model(train_data, dev_data,lr=1e-4,epochs = 4, batch_size=32,accumulation=2, bert='klue/bert-base', save=True, path='bert_qa'):
    gc.collect()
    torch.cuda.empty_cache()

    global tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(bert)

    global model,device
    model = BertForQuestionAnswering.from_pretrained(bert)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    train_dataloader,train_delete = qa_preprocess(train_data, batch_size=batch_size//accumulation, method = 'train')
    val_dataloader,val_delete = qa_preprocess(dev_data, batch_size=batch_size//accumulation, method = 'valid')
    print('')
    print('Preprocess Compelete')
    print('')

    optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)

    total_steps = len(train_dataloader) * epochs

    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)

    print('')
    print('Model Training Start')
    print(f'Epochs : {epochs} / Learning Rate : {str(lr)} / Batch Size : {batch_size}')
    print('')

    loss = []
    f1 = []
    for epoch in range(1,epochs+1):
        gc.collect()
        torch.cuda.empty_cache()
        print(f"epoch = {epoch}")

        train_lossloss = train_one_epoch(optimizer, scheduler, dataloader=train_dataloader,accumulation=2)
        preds = evaluate_one_epoch(dataloader=val_dataloader,accumulation=2)

        valid_f1 = f1_score(dev_data.drop(val_delete,axis=0)['answers'].tolist(),preds,average='micro')
        f1.append(valid_f1)

        print(f'epoch : {epoch} / loss : {train_lossloss} / f1_score : {valid_f1}')

        if save:
            model.save_pretrained(f'models/{path}')
            tokenizer.save_pretrained(f'models/{path}')
            print('Model Save')
    
    print('')
    print("Training Complete!")

    return {'loss':loss,'f1 score':f1}

In [ ]:
score,preds = qa_model(train_df, valid_df, lr = 1e-5,epochs=4,batch_size=32,accumulation=2, bert='klue/bert-base', path='qa_test')

# Test

In [13]:
def QA_test(test_data, batch_size=32,bert='QA'):
    global tokenizer
    tokenizer = BertTokenizer.from_pretrained(f'klue/bert-base')
    test_dataloader = qa_preprocess(test_data, batch_size=batch_size, method='test')

    global model, device
    model = BertForQuestionAnswering.from_pretrained(f'models/{bert}')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    answer = evaluate_one_epoch(dataloader=test_dataloader)

    new_df = valid_df[['contexts','questions']].copy()
    new_df['answers'] = answer
    
    return  new_df

In [ ]:
df = QA_test(valid_df, batch_size=32,bert='qa_test')

In [ ]:
sample = df.sample(5)

for i in range(len(sample)):
    print('context : ', sample['contexts'].tolist()[i])
    print('questions : ', sample['questions'].tolist()[i])
    print('answers : ', sample['answers'].tolist()[i])
    print('')

context :  2017년 2월 13일 오전 9시경 김정남은 말레이시아 세팡에 위치한 쿠알라룸푸르 국제공항에서 무언가를 맨손에 묻힌 여성 2명이 얼굴을 문지르고 얼마 후 사망하였다. 보툴리눔으로 죽었다는 설도 나왔고, 독극물 스프레이를 맞고 사망하였다는 설도 있다. 또한 일부는 독침으로 죽었다는 가설도 있다고 한다. 김정남 암살은 김정은이 권력을 승계받은 2011년 다음해 인 2012년 초 부터 진행되었으며, 정찰총국 등 북한 정보당국의 최순위 목표였다고 국가정보원이 밝혔다. 또한 국가정보원은 2012년 4월 김정남이 김정은 위원장에게 “살려달라”는 내용의 서신을 보낸 사실도 공개하였다. 2017년 2월 19일 말레이시아 경찰은 공식 브리핑을 통해 암살을 실행한 인도네시아,베트남 출신 여성 2명을 배후에서 사주한 일당 4명이 모두 조선민주주의인민공화국 국적이라고 발표하였다.
questions :  김정남을 살해한 배후와 사주한 일당의 국적은?
answers :  조선민주주의인민공화국

context :  1977년 입사하여 원액2과에서 7년간 근무하고 1983년 퇴사한 김봉환은 1990년 10월 30일 초진으로 이황화탄소 중독판정을 받고 회사에 산재요양신청을 냈으나 거부당했다. 1991년 1월 5일 김봉환은 직업병 증세인 정신분열로 인해 사망하였다. 김봉환이 활동하던 '원진직업병피해노동자협의회'(이하 원노협)는 이황화탄소 중독 여부를 검진 받지 못하고 사망한 데에 대한 책임을 물어 사업주를 처벌하고 부검 담당 검사에게 의뢰하여 원진직업병으로 인한 사망 여부를 판정해 줄 것을 노동부 측에 요구하였다. 그러나 병리학 검사를 의뢰받은 고려대학교의 비협조적인 태도와 회사와 사측 추천 의사들의 무성의로 사건을 제대로 마무리 짓지 못하였다. 유족과 원노협 회원들은 영결식을 위한 회사 출입을 봉쇄당하여 회사 정문 앞에서 시신투쟁을 벌이고 평일에는 대책위와 조합원을 중심으로 주말에는 수도권의 노동자와 사회단체, 학생들의 지지와 지원으로 연대집회를 열어 거리정치투쟁을 하였